**Введение:** для задачи коррекции правописания обратилась к проекту SberDevices **SAGE**. Использовала две открытые предобученные генеративные модели для коррекции правописания на русском:

*   sage-fredt5-large
*   sage-fredt5-distilled-95m

[Github:](https://github.com/ai-forever/sage?tab=readme-ov-file)

**Статья на Habr:** https://habr.com/ru/companies/sberdevices/articles/763932/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Необходимые установки**

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install pdfminer.six

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 24.0 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706


In [ ]:
# Для считывания PDF
import PyPDF2

# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure, LTTextLine

# Для извлечения текста из таблиц в PDF
import pdfplumber

from PyPDF2 import PdfReader

import re

import pandas as pd

import csv

In [ ]:
! git clone https://github.com/ai-forever/sage.git

fatal: destination path 'sage' already exists and is not an empty directory.


In [ ]:
cd sage

/content/sage


In [ ]:
! pip install .
! pip install -e .[errant]

Processing /content/sage
  Preparing metadata (setup.py) ... done
Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from sage==1.1.0)
augmentex is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Created wheel for sage: filename=sage-1.1.0-py3-none-any.whl size=47180 sha256=0e7923defc8e33c012a3f4ed0242d99752d658115fd74b76442c3eba72e476a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-zi7cc_bz/wheels/f6/fb/b3/1a788356f44a49d04bc8fdc67921c8b7f36cef909ee0b9615e
Successfully built sage
  Attempting uninstall: sage
    Found existing installation: sage 1.1.0
    Uninstalling sage-1.1.0:
      Successfully uninstalled sage-1.1.0
Obtaining file:///content/sage
  Preparing metadata (setup.py) ... done
Processing ./wheels/augmentex-1.0.3-py3-none-any.whl (from sage==1.1.0)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 3.2 MB/s eta 0:00:00
  Cloning https://github.com/Askinkaty/errant/ (to revision

In [ ]:
import os
import torch
from sage.spelling_correction import T5ModelForSpellingCorruption, RuM2M100ModelForSpellingCorrection, AvailableCorrectors

# **Текст учебника**

## ***Весь текст учебника***

In [ ]:
reader = PdfReader('/content/drive/MyDrive/Internship_2024/Copy of 7_биология_Константинов_расп.pdf')
number_of_pages = len(reader.pages)

In [ ]:
all_text = [(reader.pages[page]).extract_text() for page in range(number_of_pages)]

In [ ]:
all_text[4]

'Глава 1 ____________\nО бщ ие с в е д е н и я  о м и р е ж и в о т н ы х\n1\nЗоология — наука о ж ивотны х\nВы узнаете:\n• какие науки входят в состав зоологии;\n• каковы общие свойства объектов, изучаемых зоологией;\n• по каким признакам животные отличаются от растений.\nВспомните\n• Каковы общие признаки растений?\nВведение\nВ предыдущ их классах вы познакомились с представителями трёх \nцарств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению \nнового царства живых организмов — Животные.\nРаздел биологии, посвящённый изучению животных, их разнообразия, \nстроения и ж изнедеятельности, связей со средой обитания, распростране\xad\nния, индивидуального и исторического развития, роли в природе и зн а\xad\nчения для человека, назы вается зоологи ей  (от греч. зоон  — «животное», \nлогос  — «учение»).\nСовременная зоология —  это система наук. Например, м орф ологи я  \nи анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга\xad\nнизмов. Ф и зи ол оги я ж ивот ны

In [ ]:
def preprocessing(text): # Минимальный препроцессинг
  text_clean = [item.replace('\xad', '') for item in text]
  text_clean = [item.replace('\n', ' ') for item in text_clean]
  return text_clean

In [ ]:
all_text_clean0 = preprocessing(all_text) # Предобработаем текст

## ***Глава 1, параграф 1***

In [ ]:
chapter_1 = [(reader.pages[ch]).extract_text() for ch in range(4, 9)] # извлечем текст параграфа 1

In [ ]:
chapter_1

['Глава 1 ____________\nО бщ ие с в е д е н и я  о м и р е ж и в о т н ы х\n1\nЗоология — наука о ж ивотны х\nВы узнаете:\n• какие науки входят в состав зоологии;\n• каковы общие свойства объектов, изучаемых зоологией;\n• по каким признакам животные отличаются от растений.\nВспомните\n• Каковы общие признаки растений?\nВведение\nВ предыдущ их классах вы познакомились с представителями трёх \nцарств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению \nнового царства живых организмов — Животные.\nРаздел биологии, посвящённый изучению животных, их разнообразия, \nстроения и ж изнедеятельности, связей со средой обитания, распростране\xad\nния, индивидуального и исторического развития, роли в природе и зн а\xad\nчения для человека, назы вается зоологи ей  (от греч. зоон  — «животное», \nлогос  — «учение»).\nСовременная зоология —  это система наук. Например, м орф ологи я  \nи анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга\xad\nнизмов. Ф и зи ол оги я ж ивот н

In [ ]:
chapter_1_clean = preprocessing(chapter_1) # предобработаем

In [ ]:
chapter_1_clean_joined = [' '.join(chapter_1_clean)] # внутри списка объединим строки, чтобы получился список, длиной 1, затем скормим модели и посмоттрим результат

In [ ]:
chapter_1_clean_joined

['Глава 1 ____________ О бщ ие с в е д е н и я  о м и р е ж и в о т н ы х 1 Зоология — наука о ж ивотны х Вы узнаете: • какие науки входят в состав зоологии; • каковы общие свойства объектов, изучаемых зоологией; • по каким признакам животные отличаются от растений. Вспомните • Каковы общие признаки растений? Введение В предыдущ их классах вы познакомились с представителями трёх  царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению  нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия,  строения и ж изнедеятельности, связей со средой обитания, распростране ния, индивидуального и исторического развития, роли в природе и зн а чения для человека, назы вается зоологи ей  (от греч. зоон  — «животное»,  логос  — «учение»). Современная зоология —  это система наук. Например, м орф ологи я   и анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга низмов. Ф и зи ол оги я ж ивот ны х  исследует деятельность клето

# **sage-fredt5-large**

In [ ]:
sage_fredt5_large_corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_mt5_large.value)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## ***Применение***

In [ ]:
all_text_clean0[4] # из всего текста учебника возьмем фрагмент под индексом 4 (длина строки 1517)

'Глава 1 ____________ О бщ ие с в е д е н и я  о м и р е ж и в о т н ы х 1 Зоология — наука о ж ивотны х Вы узнаете: • какие науки входят в состав зоологии; • каковы общие свойства объектов, изучаемых зоологией; • по каким признакам животные отличаются от растений. Вспомните • Каковы общие признаки растений? Введение В предыдущ их классах вы познакомились с представителями трёх  царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению  нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия,  строения и ж изнедеятельности, связей со средой обитания, распростране ния, индивидуального и исторического развития, роли в природе и зн а чения для человека, назы вается зоологи ей  (от греч. зоон  — «животное»,  логос  — «учение»). Современная зоология —  это система наук. Например, м орф ологи я   и анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга низмов. Ф и зи ол оги я ж ивот ны х  исследует деятельность клеток

In [ ]:
result_large0 = sage_fredt5_large_corrector.correct(all_text_clean0[4])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_large0

['Глава 1 ____________ О б и е с в е д е н и я о м и р е ж и в о т н ы х. 1 Зоология — наука о животных — Вы узнаете: \xad какие науки входят в состав зоологии; \xad каковы общие свойства объектов, изучаемых зоологией; \xad по каким признакам животные отличаются от растений. Вспомните: \xad каковы общие признаки растений? Введение: В предыдущих их классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и их жизнедеятельности, связей со средой обитания, распространения их индивидуального и исторического развития, роли в природе и т. д. Учения для человека, называется зоологией (от греч. зоон — «животное», логос — «учение»). Современная зоология — это система наук. Например, м о ология я и а н а и я и вот ны х изучают и внешнее и внутреннее строение органов и м и о логи я и вот ны х изучают и взаимоотношения живо

In [ ]:
len(result_large0[0])

1375

**Попробую применить модель на всем тексте параграфа 1**

In [ ]:
result_large_ch_1 = sage_fredt5_large_corrector.correct(chapter_1_clean_joined[0])  # применим модель к минимально предобр. тексту параграфа 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
chapter_1_clean_joined[0]

'Глава 1 ____________ О бщ ие с в е д е н и я  о м и р е ж и в о т н ы х 1 Зоология — наука о ж ивотны х Вы узнаете: • какие науки входят в состав зоологии; • каковы общие свойства объектов, изучаемых зоологией; • по каким признакам животные отличаются от растений. Вспомните • Каковы общие признаки растений? Введение В предыдущ их классах вы познакомились с представителями трёх  царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению  нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия,  строения и ж изнедеятельности, связей со средой обитания, распростране ния, индивидуального и исторического развития, роли в природе и зн а чения для человека, назы вается зоологи ей  (от греч. зоон  — «животное»,  логос  — «учение»). Современная зоология —  это система наук. Например, м орф ологи я   и анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга низмов. Ф и зи ол оги я ж ивот ны х  исследует деятельность клеток

In [ ]:
result_large_ch_1 # очень странный результат на большом фрагменте.

['Глава 1 ____________ О б и е с в е д е н и я о м и р е ж и в о т н ы х. 1 Зоология — наука о животных. Вы узнаете: * какие науки входят в состав зоологии; * каковы общие свойства объектов, изучаемых зоологией; * способность к различению животных и растений. Для животных, как и для большинства других живых организмов, их изучают и взаимодействующие клетки, и в организме их организмов, и в их среде. Разнообразие животных и растений. Для животных, как и для других видов животных и их организмов, их различают по облику и строению, размерам, образу жизни. Зоология изучает различные виды животных и растений. Для животных, как и для других видов животных и существ, их организмы и их организмы, их животные, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их организмы, их органи

**Попробую по одному элементу (их всего 5) извлеченного первого параграфа**

In [ ]:
# 1

In [ ]:
chapter_1_clean[0]

'Глава 1 ____________ О бщ ие с в е д е н и я  о м и р е ж и в о т н ы х 1 Зоология — наука о ж ивотны х Вы узнаете: • какие науки входят в состав зоологии; • каковы общие свойства объектов, изучаемых зоологией; • по каким признакам животные отличаются от растений. Вспомните • Каковы общие признаки растений? Введение В предыдущ их классах вы познакомились с представителями трёх  царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению  нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия,  строения и ж изнедеятельности, связей со средой обитания, распростране ния, индивидуального и исторического развития, роли в природе и зн а чения для человека, назы вается зоологи ей  (от греч. зоон  — «животное»,  логос  — «учение»). Современная зоология —  это система наук. Например, м орф ологи я   и анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга низмов. Ф и зи ол оги я ж ивот ны х  исследует деятельность клеток

In [ ]:
result_ch_1_1 = sage_fredt5_large_corrector.correct(chapter_1_clean[0])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_ch_1_1

['Глава 1 ____________ О б и е с в е д е н и я о м и р е ж и в о т н ы х. 1 Зоология — наука о животных — Вы узнаете: \xad какие науки входят в состав зоологии; \xad каковы общие свойства объектов, изучаемых зоологией; \xad по каким признакам животные отличаются от растений. Вспомните: \xad каковы общие признаки растений? Введение: В предыдущих их классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и их жизнедеятельности, связей со средой обитания, распространения их индивидуального и исторического развития, роли в природе и т. д. Учения для человека, называется зоологией (от греч. зоон — «животное», логос — «учение»). Современная зоология — это система наук. Например, м о ология я и а н а и я и вот ны х изучают и внешнее и внутреннее строение органов и м и о логи я и вот ны х изучают и взаимоотношения живо

In [ ]:
# 2 В данном пункте нет надобности применять генеративную модель

In [ ]:
chapter_1_clean[1]

'Рис. 1. Разнообразие многоклеточных животных: А — беспозвоночные; Б — позвоночные 5 '

In [ ]:
# 3

In [ ]:
chapter_1_clean[2]

'Рис. 2. Разнообразие одноклеточных животных (простейших) Сходство и различие животных и растений Для животных, как и для большинства других ж ивых организмов, ха рактерны следующие особенности: клеточное строение, способность к пи танию, дыханию, выделению, наличие обмена веществ меж ду организмом  и окружаю щ ей средой, способность к размножению, росту, развитию. Ж ивотные отличаются от растений по следующим признакам. Клетки  животных не имеют твёрдой целлюлозной оболочки. В отличие от расте ний животные питаю тся готовыми органическими веществами. В природ ных сообществах они играют роль потребителей органического вещества.  Ж ивотные способны воспринимать раздраж ения и реагировать на них.  Они могут активно передвигаться. Большинство сами добывают себе пищу,  преследуют добычу. Разнообразие животных По данным Всемирного (М еждународного) союза охраны природы I  (ВСОП), на Земле сущ ествует более 1,2 млн видов животных. Больш е все- I  го насекомых — около 1  млн видов. Известно о

In [ ]:
result_ch_1_3 = sage_fredt5_large_corrector.correct(chapter_1_clean[2])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_ch_1_3

['Рис. 2. Разнообразие одноклеточных животных (простейших). Сходство и различие животных и растений. Для животных, как и для большинства других живых организмов, их характерны следующие особенности: клеточное строение, способность к питанию, дыханию, выделению, наличие обмена веществ между их организмом и окружающей их ей средой, способность к размножению, росту, развитию. Животные отличаются от растений по следующим признакам. Клетки животных не имеют твёрдой целлюлозной оболочки. В отличие от растений, живые животные питаются готовыми органическими веществами. В природе и сообществах они играют роль потребителей органического вещества. Животные способны воспринимать раздражения и реагировать на них. Они могут активно передвигаться. Большинство сами добывают себе пищу, преследуют добычу. Разнообразие животных. По данным Всемирного ( ) Международного союза охраны природы I (ВСОП), на Земле уже существует более 1,2 млн видов животных. Больше I всех I, но насекомых около 1 млн видов. Изв

In [ ]:
# 4

In [ ]:
chapter_1_clean[3]

'Рис. 3. Сравнительные размеры животных (простейшие даны в поле оптического  микроскопа) Самых маленьких животных можно увидеть только под микроскопом.  Крупные наземные животные, например слоны, достигают в высоту 3,5 м,  а масса их тела около 5 т. Самое большое животное из когда-либо живш их  и живущих на Земле —  синий кит длиной более 30 м и массой до 150 т (рис. 3). Значение животных Ж ивотные различаю тся не только по внешнему облику, строению, об разу ж изни, но и по той роли, которую  они играю т в природны х сооб ществах. Велика роль в природе животных — опы лит елей  растений  (рис. 4). Это бабочки, жуки, мухи, шмели, пчёлы и другие организмы. Без  них облик наш их лесов, лугов, полей был бы совсем другим. Многие ж ивот ные распространяют плоды и семена растений, перенося их на шерсти  и перьях или питаясь сочными плодами. М якоть переваривается, а семена  проходят через киш ечник и разносятся на большие расстояния. Некоторые животные играют отрицательную роль в природных соо

In [ ]:
result_ch_1_4 = sage_fredt5_large_corrector.correct(chapter_1_clean[3])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_ch_1_4

['Рис. 3. Сравнительные размеры животных (простейшие) даны в поле оптического микроскопа. Самых маленьких животных можно увидеть только под микроскопом. Крупные наземные животные, например, слоны, достигают в высоту 3,5 м, а масса их тела около 5 т. Самое большое животное из когда-либо живших и не живущих на Земле — синий кит длиной более 30 м и массой до 150 т (рис. 3). Значение животных. Животные различаются не только по внешнему облику, строению и образу жизни, но и по той роли, которую они играют и в природных их сообществах. Велика роль в природе животных — осы или тлей растений (рис. 4). Это бабочки, жуки, мухи, шмели, пчёлы и другие организмы. Без них облик наших лесов, лугов, полей был бы совсем другим. Многие же животные распространяют плоды и семена растений, перенося их на шерсти и перья или питаясь сочными плодами. Мякоть переваривается, а семена проходят через их яичники и разносятся на большие расстояния. Некоторые животные играют отрицательную роль в природных их сообщес

In [ ]:
# 5

In [ ]:
chapter_1_clean[4]

'Рис. 4. Насекомые — опылители растений азиатской перелётной саранчи (рис. 5) уничтожаю т растительность на  огромных площ адях, лиш ая пищи другие растительноядные организмы.  Есть животные, вызываю щ ие болезни у растений, других животных и че ловека. Отдельные животные, наоборот, снижают число вредителей лесных  и культурных растений, питаясь ими. Большую ценность для человека имеют дикие животные как объекты  охоты. Значительную  часть белковой пищи человек получает за счёт вы лова рыбы в морях и океанах, промысла морского зверя, охоты на каба нов, оленей, водоплавающих и лесных птиц. Исключительно важное значение  для людей имеют домашние живот ные. Мы получаем от них продукты  питания (молоко, яйца, мёд), шерсть,  пух, кожу, шёлк. Домашние ж ивот ные выполняю т различную  работу  в сельском хозяйстве, служ ат транс портными средствами. Декоративные  животные ж ивут вместе с человеком  в квартирах и домах, скраш ивая до суг горожан. Больш инство одомаш-  Рис. 5. Азиатская перелётн

In [ ]:
result_ch_1_5 = sage_fredt5_large_corrector.correct(chapter_1_clean[4])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_ch_1_5

['Рис. 4. Насекомые и опылители растений азиатской перелётной саранчи (рис. 5) уничтожают и растительность на огромных площадях, даже для пищи другие растительноядные организмы. Есть животные, вызывающие и те болезни у растений, других животных и у человека. Отдельные животные, наоборот, снижают число вредителей лесных и культурных растений, питаясь ими. Большую ценность для человека имеют дикие животные как объекты охоты. Значительную часть белковой пищи человек получает за счёт воды, лова рыбы в морях и океанах, промысла морского зверя, охоты на кабанов, коров, оленей, водоплавающих и лесных птиц. Исключительно важное значение для людей имеют домашние животные - мы получаем от них продукты питания (молоко, яйца, мёд), шерсть, пух, кожу, шёлк. Домашние же животные выполняют и различную работу в сельском хозяйстве, служат нам транспортными средствами. Декоративные животные - живут вместе с человеком в квартирах и домах, спрашивая о друге горожан. Большинство домов. Рис. 5. Азиатская пе

In [ ]:
# Объединим все фрагменты в один список

In [ ]:
ch_1_result = result_ch_1_1 + [chapter_1_clean[1]] + result_ch_1_3 + result_ch_1_4 + result_ch_1_5

In [ ]:
ch_1_result

['Глава 1 ____________ О б и е с в е д е н и я о м и р е ж и в о т н ы х. 1 Зоология — наука о животных — Вы узнаете: \xad какие науки входят в состав зоологии; \xad каковы общие свойства объектов, изучаемых зоологией; \xad по каким признакам животные отличаются от растений. Вспомните: \xad каковы общие признаки растений? Введение: В предыдущих их классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и их жизнедеятельности, связей со средой обитания, распространения их индивидуального и исторического развития, роли в природе и т. д. Учения для человека, называется зоологией (от греч. зоон — «животное», логос — «учение»). Современная зоология — это система наук. Например, м о ология я и а н а и я и вот ны х изучают и внешнее и внутреннее строение органов и м и о логи я и вот ны х изучают и взаимоотношения живо

In [ ]:
ch_1_result_fin = [' '.join(ch_1_result)] # внутри списка объединим строки, чтобы получился список, длиной 1 и передадим в DataFrame

In [ ]:
ch_1_result_fin

['Глава 1 ____________ О б и е с в е д е н и я о м и р е ж и в о т н ы х. 1 Зоология — наука о животных — Вы узнаете: \xad какие науки входят в состав зоологии; \xad каковы общие свойства объектов, изучаемых зоологией; \xad по каким признакам животные отличаются от растений. Вспомните: \xad каковы общие признаки растений? Введение: В предыдущих их классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и их жизнедеятельности, связей со средой обитания, распространения их индивидуального и исторического развития, роли в природе и т. д. Учения для человека, называется зоологией (от греч. зоон — «животное», логос — «учение»). Современная зоология — это система наук. Например, м о ология я и а н а и я и вот ны х изучают и внешнее и внутреннее строение органов и м и о логи я и вот ны х изучают и взаимоотношения живо

# **sage-fredt5-distilled-95m**

In [ ]:
sage_fredt5_95m_corrector = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.sage_fredt5_distilled_95m.value)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/383M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.72k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## ***Применение***

**Попробую применить модель на всем тексте параграфа 1**

In [ ]:
chapter_1_clean_joined[0] # Мин. предобр.текст параграфа №1

'Глава 1 ____________ О бщ ие с в е д е н и я  о м и р е ж и в о т н ы х 1 Зоология — наука о ж ивотны х Вы узнаете: • какие науки входят в состав зоологии; • каковы общие свойства объектов, изучаемых зоологией; • по каким признакам животные отличаются от растений. Вспомните • Каковы общие признаки растений? Введение В предыдущ их классах вы познакомились с представителями трёх  царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению  нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия,  строения и ж изнедеятельности, связей со средой обитания, распростране ния, индивидуального и исторического развития, роли в природе и зн а чения для человека, назы вается зоологи ей  (от греч. зоон  — «животное»,  логос  — «учение»). Современная зоология —  это система наук. Например, м орф ологи я   и анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга низмов. Ф и зи ол оги я ж ивот ны х  исследует деятельность клеток

In [ ]:
result_95m = sage_fredt5_95m_corrector.correct(chapter_1_clean_joined[0])  # применим модель к минимально предобр. тексту параграфа 1

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_95m # Результат работы модели

['Глава 1. Объёмы и вес в еде, дё н и я, о м и р е ж и в о т н ы х 1. Зоология — наука о животных. Вы узнаете, какие науки входят в состав зоологии; каковы общие свойства объектов, изучаемых зоологией; по каким признакам животные отличаются от растений. Вспомните, каковы общие признаки растений. Введение в предыдущих классах вы познакомились с представителями трёх царств: бактерии, грибы, растения. Теперь вы приступаете к изучению нового царства живых организмов — животных. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и жизнедеятельности, связей со средой обитания, распространения индивидуального и исторического развития, роли в природе и зн. ачения для человека, называют зоологи ей (от греч. зоон  — "животное", этология — "опылители").  - это система наук. Например, м орфологии я и животных животных  - исследуют деятельность клеток, органов, систем органов и целых организмов. \nЕсть животные, имеют различную работу в сельском хозяйстве,  - это млекопитающи

**Попробую по одному элементу (их всего 5) извлеченного первого параграфа**

In [ ]:
# 1

In [ ]:
chapter_1_clean[0]

'Глава 1 ____________ О бщ ие с в е д е н и я  о м и р е ж и в о т н ы х 1 Зоология — наука о ж ивотны х Вы узнаете: • какие науки входят в состав зоологии; • каковы общие свойства объектов, изучаемых зоологией; • по каким признакам животные отличаются от растений. Вспомните • Каковы общие признаки растений? Введение В предыдущ их классах вы познакомились с представителями трёх  царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению  нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия,  строения и ж изнедеятельности, связей со средой обитания, распростране ния, индивидуального и исторического развития, роли в природе и зн а чения для человека, назы вается зоологи ей  (от греч. зоон  — «животное»,  логос  — «учение»). Современная зоология —  это система наук. Например, м орф ологи я   и анат ом ия ж ивот ны х  изучаю т внешнее и внутреннее строение орга низмов. Ф и зи ол оги я ж ивот ны х  исследует деятельность клеток

In [ ]:
result_95_ch_1_1 = sage_fredt5_95m_corrector.correct(chapter_1_clean[0])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_95_ch_1_1

['Глава 1. Об общ. ие с в е д е н и я о м и р е ж и в о т н ы х 1. Зоология — наука о животных. Вы узнаете: какие науки входят в состав зоологии; каковы общие свойства объектов, изучаемых зоологией; по каким признакам животные отличаются от растений. Вспомните: каковы общие признаки растений? Введение: в предыдущих классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и жизнедеятельности, связей со средой обитания, распространения индивидуального и исторического развития, роли в природе и зн. ачения для человека, называется зоологией (от греч. зоон  — "животное", логос  — "учение"). Современная зоология — это система наук. Например, м орфологии я и анат о м и я ж живот ны х  изучаю и внешнее и внутреннее строение органов, низмов. Ф и з и ол оги я ж живот ны х  исследует деятельность клеток, органов, систем ор

In [ ]:
# 3

In [ ]:
chapter_1_clean[2]

'Рис. 2. Разнообразие одноклеточных животных (простейших) Сходство и различие животных и растений Для животных, как и для большинства других ж ивых организмов, ха рактерны следующие особенности: клеточное строение, способность к пи танию, дыханию, выделению, наличие обмена веществ меж ду организмом  и окружаю щ ей средой, способность к размножению, росту, развитию. Ж ивотные отличаются от растений по следующим признакам. Клетки  животных не имеют твёрдой целлюлозной оболочки. В отличие от расте ний животные питаю тся готовыми органическими веществами. В природ ных сообществах они играют роль потребителей органического вещества.  Ж ивотные способны воспринимать раздраж ения и реагировать на них.  Они могут активно передвигаться. Большинство сами добывают себе пищу,  преследуют добычу. Разнообразие животных По данным Всемирного (М еждународного) союза охраны природы I  (ВСОП), на Земле сущ ествует более 1,2 млн видов животных. Больш е все- I  го насекомых — около 1  млн видов. Известно о

In [ ]:
result_95_ch_1_3 = sage_fredt5_95m_corrector.correct(chapter_1_clean[2])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_95_ch_1_3

['Рис. 2. Разнообразие одноклеточных животных (простейших). Сходство и различие животных и растений. Для животных, как и для большинства других живых организмов, характерны следующие особенности: клеточное строение, способность к питанию, дыханию, выделению, наличие обмена веществ между организмом и окружающей ей средой, способность к размножению, росту, развитию. Животные отличаются от растений по следующим признакам. Клетки животных не имеют твёрдой целлюлозной оболочки. В отличие от растений, животные питаются готовыми органическими веществами. В природных сообществах они играют роль потребителей органического вещества. Животные способны воспринимать раздражения и реагировать на них. Они могут активно передвигаться. Большинство сами добывают себе пищу, преследуют добычу. Разнообразие животных по данным Всемирного (Международного) союза охраны природы I (ВСОП) на Земле существует более 1,2 млн видов животных. Больше всех I его насекомых — около 1 млн видов. Известно около 70 тыс. вид

In [ ]:
# 4

In [ ]:
chapter_1_clean[3]

'Рис. 3. Сравнительные размеры животных (простейшие даны в поле оптического  микроскопа) Самых маленьких животных можно увидеть только под микроскопом.  Крупные наземные животные, например слоны, достигают в высоту 3,5 м,  а масса их тела около 5 т. Самое большое животное из когда-либо живш их  и живущих на Земле —  синий кит длиной более 30 м и массой до 150 т (рис. 3). Значение животных Ж ивотные различаю тся не только по внешнему облику, строению, об разу ж изни, но и по той роли, которую  они играю т в природны х сооб ществах. Велика роль в природе животных — опы лит елей  растений  (рис. 4). Это бабочки, жуки, мухи, шмели, пчёлы и другие организмы. Без  них облик наш их лесов, лугов, полей был бы совсем другим. Многие ж ивот ные распространяют плоды и семена растений, перенося их на шерсти  и перьях или питаясь сочными плодами. М якоть переваривается, а семена  проходят через киш ечник и разносятся на большие расстояния. Некоторые животные играют отрицательную роль в природных соо

In [ ]:
result_95_ch_1_4 = sage_fredt5_95m_corrector.correct(chapter_1_clean[3])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_95_ch_1_4

['Рис. 3. Сравнительные размеры животных (простейшие даны в поле оптического микроскопа). Самых маленьких животных можно увидеть только под микроскопом. Крупные наземные животные, например слоны, достигают в высоту 3,5 м, а масса их тела около 5 т. Самое большое животное из когда-либо живших и живущих на Земле — синий кит длиной более 30 м и массой до 150 т (рис. 3). Значение животных. Животные различаются не только по внешнему облику, строению, образу жизни, но и по той роли, которую они играют в природных сообществах. Велика роль в природе животных — опы лит елей растений (рис. 4). Это бабочки, жуки, мухи, шмели, пчёлы и другие организмы. Без них облик наших лесов, лугов, полей был бы совсем другим. Многие животные распространяют плоды и семена растений, перенося их на шерсти и перьях или питаясь сочными плодами. Мякьь переваривается, а семена проходят через киш, ечник и разносятся на большие расстояния. Некоторые животные играют отрицательную роль в природных сообществах. Массовое п

In [ ]:
# 5

In [ ]:
chapter_1_clean[4]

'Рис. 4. Насекомые — опылители растений азиатской перелётной саранчи (рис. 5) уничтожаю т растительность на  огромных площ адях, лиш ая пищи другие растительноядные организмы.  Есть животные, вызываю щ ие болезни у растений, других животных и че ловека. Отдельные животные, наоборот, снижают число вредителей лесных  и культурных растений, питаясь ими. Большую ценность для человека имеют дикие животные как объекты  охоты. Значительную  часть белковой пищи человек получает за счёт вы лова рыбы в морях и океанах, промысла морского зверя, охоты на каба нов, оленей, водоплавающих и лесных птиц. Исключительно важное значение  для людей имеют домашние живот ные. Мы получаем от них продукты  питания (молоко, яйца, мёд), шерсть,  пух, кожу, шёлк. Домашние ж ивот ные выполняю т различную  работу  в сельском хозяйстве, служ ат транс портными средствами. Декоративные  животные ж ивут вместе с человеком  в квартирах и домах, скраш ивая до суг горожан. Больш инство одомаш-  Рис. 5. Азиатская перелётн

In [ ]:
result_95_ch_1_5 = sage_fredt5_95m_corrector.correct(chapter_1_clean[4])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
result_95_ch_1_5

['Рис. 4. Насекомые — опылители растений азиатской перелётной саранчи (рис. 5) уничтожают растительность на огромных площадях, лишая пищи другие растительноядные организмы. Есть животные, вызывающие те болезни у растений, других животных и человека. Отдельные животные, наоборот, снижают число вредителей лесных и культурных растений, питаясь ими. Большую ценность для человека имеют дикие животные как объекты охоты. Значительную часть белковой пищи человек получает за счёт вылова рыбы в морях и океанах, промысла морского зверя, охоты на кабанов, оленей, водоплавающих и лесных птиц. Исключительно важное значение для людей имеют домашние животные ные. Мы получаем от них продукты питания (молоко, яйца, мёд), шерсть, пух, кожу, шёлк. Домашние животные живут ные, выполняют различную работу в сельском хозяйстве, служат транс портными средствами. Декоративные животные живут вместе с человеком в квартирах и домах, скрашивая до суг горожан. Большинство одомаш. Рис. 5. Азиатская перелётная ненных 

In [ ]:
# Объединим все фрагменты в один список

In [ ]:
ch_1_result_95 = result_95_ch_1_1 + [chapter_1_clean[1]] + result_95_ch_1_3 + result_95_ch_1_4 + result_95_ch_1_5

In [ ]:
ch_1_result_95

['Глава 1. Об общ. ие с в е д е н и я о м и р е ж и в о т н ы х 1. Зоология — наука о животных. Вы узнаете: какие науки входят в состав зоологии; каковы общие свойства объектов, изучаемых зоологией; по каким признакам животные отличаются от растений. Вспомните: каковы общие признаки растений? Введение: в предыдущих классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и жизнедеятельности, связей со средой обитания, распространения индивидуального и исторического развития, роли в природе и зн. ачения для человека, называется зоологией (от греч. зоон  — "животное", логос  — "учение"). Современная зоология — это система наук. Например, м орфологии я и анат о м и я ж живот ны х  изучаю и внешнее и внутреннее строение органов, низмов. Ф и з и ол оги я ж живот ны х  исследует деятельность клеток, органов, систем ор

In [ ]:
ch_1_result_95_fin = [' '.join(ch_1_result_95)] # внутри списка объединим строки, чтобы получился список, длиной 1 и передадим в DataFrame

In [ ]:
ch_1_result_95_fin

['Глава 1. Об общ. ие с в е д е н и я о м и р е ж и в о т н ы х 1. Зоология — наука о животных. Вы узнаете: какие науки входят в состав зоологии; каковы общие свойства объектов, изучаемых зоологией; по каким признакам животные отличаются от растений. Вспомните: каковы общие признаки растений? Введение: в предыдущих классах вы познакомились с представителями трёх царств: Бактерии, Грибы, Растения. Теперь вы приступаете к изучению нового царства живых организмов — Животные. Раздел биологии, посвящённый изучению животных, их разнообразия, строения и жизнедеятельности, связей со средой обитания, распространения индивидуального и исторического развития, роли в природе и зн. ачения для человека, называется зоологией (от греч. зоон  — "животное", логос  — "учение"). Современная зоология — это система наук. Например, м орфологии я и анат о м и я ж живот ны х  изучаю и внешнее и внутреннее строение органов, низмов. Ф и з и ол оги я ж живот ны х  исследует деятельность клеток, органов, систем ор

# **Извлечение наименований глав и параграфов**

In [ ]:
title_pattern = r'(Глава \d*\. *\w[^.]*)(?=§)'
title_list = []

for q in range(number_of_pages):
  current_page = reader.pages[q]
  page_text = current_page.extract_text()
  if re.findall(title_pattern, page_text):
    title_list.append(re.findall(title_pattern, page_text))

In [ ]:
title_list

[['Глава 1.  Общие сведения о мире животных\n',
  'Глава 2.  Строение тела животных\n',
  'Глава 3.  Подцарство Простейшие (Protozoa)\n',
  'Глава 4.  Тип Кишечнополостные (Coelenterata)\n',
  'Глава 5.  Типы Плоские черви (Plathelminthes),\nКруглые черви (Nemathelminthes),\nКольчатые черви (Annelida)\n'],
 ['Глава 6.  Тип Моллюски (Mollusca)\n',
  'Глава 7.  Тип Членистоногие (Arthropoda)\n',
  'Глава 8.  Тип Хордовые (Chordata): бесчерепные, рыбы\n',
  'Глава 9.  Класс Земноводные, или Амфибии (Amphibia)\n',
  'Глава 10.  Класс Пресмыкающиеся, или Рептилии (Reptilia)\n'],
 ['Глава 11.  Класс Птицы (Aves)\n',
  'Глава 12.  Класс Млекопитающие, или Звери (M ammalia)\n',
  'Глава 13.  Развитие животного мира на Земле\n']]

In [ ]:
title_list0 = [titl.replace('\n', '') for tit in title_list for titl in tit] # уберем \n и список сделаем плоским

In [ ]:
title_list0

['Глава 1.  Общие сведения о мире животных',
 'Глава 2.  Строение тела животных',
 'Глава 3.  Подцарство Простейшие (Protozoa)',
 'Глава 4.  Тип Кишечнополостные (Coelenterata)',
 'Глава 5.  Типы Плоские черви (Plathelminthes),Круглые черви (Nemathelminthes),Кольчатые черви (Annelida)',
 'Глава 6.  Тип Моллюски (Mollusca)',
 'Глава 7.  Тип Членистоногие (Arthropoda)',
 'Глава 8.  Тип Хордовые (Chordata): бесчерепные, рыбы',
 'Глава 9.  Класс Земноводные, или Амфибии (Amphibia)',
 'Глава 10.  Класс Пресмыкающиеся, или Рептилии (Reptilia)',
 'Глава 11.  Класс Птицы (Aves)',
 'Глава 12.  Класс Млекопитающие, или Звери (M ammalia)',
 'Глава 13.  Развитие животного мира на Земле']

In [ ]:
# Параграфы

In [ ]:
chapter_pattern = r'§ \d{1,2}\. \w[^.]*'
chapter_list = []

for c in range(number_of_pages):
  current_page_c = reader.pages[c]
  page_text_c = current_page_c.extract_text()
  if re.findall(chapter_pattern, page_text_c):
    chapter_list.append(re.findall(chapter_pattern, page_text_c))

In [ ]:
chapter_list0 = [chapt.replace('\n', ' ') for chap in chapter_list for chapt in chap] # поступим аналогично как с главами

In [ ]:
chapter_list0

['§ 1. Зоология — наука о ж и вотн ы х ',
 '§ 2. Ж ивотные и окружаю щ ая с р е д а ',
 '§ 3. Классификация животных и основные систематические груп п ы ',
 '§ 4. Влияние человека на ж и в о тн ы х ',
 '§ 5. К раткая история развития зоологии ',
 '§ 6. К л е тк а ',
 '§ 7. Ткани, органы и системы органов ',
 '§ 8. Тип Саркодовые и жгутиконосцы (Sarcom astigophora)',
 '§ 9. Тип Саркодовые и жгутиконосцы',
 '§ 10. Тип И нфузории (C iliophora) ',
 '§ 11. Значение простейш их ',
 '§ 12. Строение и ж изнедеятельность киш ечнополостных',
 '§ 13. Разнообразие киш ечнополостных ',
 '§ 14. Тип Плоские черви (P lath elm in th es) ',
 '§ 15. Разнообразие плоских червей: сосальщ ики и цепни ',
 '§ 16. Тип Круглые черви (N em ath elm in th es) ',
 '§ 17. Тип Кольчатые черви (Annelida)',
 '§ 18. Тип Кольчатые черви (Annelida)',
 '§ 19. Общая характеристика м оллю сков ',
 '§ 20. Класс Брюхоногие моллюски (G astropoda) ',
 '§ 21. Класс Двустворчатые моллюски (B ivalvia) ',
 '§ 22. Класс Головоногие м

In [ ]:
# Применим модель, чтобы объединила слова
sage_5_large_ch_name = sage_fredt5_large_corrector.correct(chapter_list0[0])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sage_5_large_ch_name

['P.S. 1. Зоология наука о лжи и вот о х']

In [ ]:
# Применим модель sage-fredt5-distilled-95m, чтобы объединила слова
sage_95m_ch_name = sage_fredt5_95m_corrector.correct(chapter_list0[0])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sage_95m_ch_name

['Ч. 1. Зоология — наука о ж и вотны х. ']

# **Ключевые слова (по всему тексту)**

## *Отдельная страница параграфа*

Если хотим извлечь ключевые слова с конкретной страницы (как правило, последняя страница параграфа), пользуемся данным кодом, вставляя нужную страницу.

In [ ]:
#извлечение слов, выделенных курсивом, на последней стр. параграфа №2
key_word_ch_2 = []
with pdfplumber.open('/content/drive/MyDrive/Internship_2024/Copy of 7_биология_Константинов_расп.pdf') as pdf_bio2:
  key_page_ch_2 = pdf_bio2.pages[15]
  key_wd_ch_2 = key_page_ch_2.filter(lambda obj2: obj2["object_type"] == "char" and "Italic" in obj2["fontname"]) #TimesNewRomanPS-BoldItalicMT
  key_word_ch_2.append(key_wd_ch_2.extract_text())
#print(*key_word_ch_2, end = '\n')

In [ ]:
key_word_ch_2

['Среды жизни (наземно-воздушная, водная, почва, организм),\nместа обитания, экологические факторы (биотические, абио\xad\nтические, антропогенные), среда обитания хищничество,\nпаразиты и хозяева, конкурентные отношения, биоценозы,\nпищевые связи, цепи питания.']

In [ ]:
# Можно минимально предобработать

In [ ]:
key_word_ch_2_prep = [wd2.replace('\n', ' ') for wd2 in key_word_ch_2]
key_word_ch_2_prep = [wd2.replace('\xad', '') for wd2 in key_word_ch_2_prep]
#key_word_2_last_cl = [wd2.replace('абио тические', 'абиотические') for wd2 in key_word_2_last_cl]

In [ ]:
key_word_ch_2_prep

['Среды жизни (наземно-воздушная, водная, почва, организм), места обитания, экологические факторы (биотические, абио тические, антропогенные), среда обитания хищничество, паразиты и хозяева, конкурентные отношения, биоценозы, пищевые связи, цепи питания.']

In [ ]:
sage_5_large_ch_2 = sage_fredt5_large_corrector.correct(key_word_ch_2_prep[0]) #применим модель, чтобы объединила слово абиотические

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sage_5_large_ch_2 # модель сработала плохо - вместо "абиотеские" получили на выходе "био тические"

['Среды жизни (наземно-воздушная, водная почва, организм), места обитания, экологические факторы (биотические, био тические, антропогенные), среда обитания (хищничество, паразиты и хозяева), конкурентные отношения, биоценозы, пищевые связи, цепи питания.']

## *Все страницы учебника*

Вытащим все слова, выделенные курсивом

In [ ]:
dict_word = {}
with pdfplumber.open('/content/drive/MyDrive/Internship_2024/Copy of 7_биология_Константинов_расп.pdf') as pdf:
  for pdf_pg in range(number_of_pages):
    key_page = pdf.pages[pdf_pg]
    key_word = key_page.filter(lambda obj: obj["object_type"] == "char" and "Italic" in obj["fontname"]) #TimesNewRomanPS-BoldItalicMT
    dict_word[key_page] = key_word.extract_text()

In [ ]:
dict_word

{<Page:1>: '',
 <Page:2>: 'издание, переработанное',
 <Page:3>: '',
 <Page:4>: 'полужирным курсивом.',
 <Page:5>: 'Глава 1\nзоологией зоон\nлогос\nморфология\nанатомия животных\nФизиология животных\nэкология животных,\nПалеонтология\nЭтология',
 <Page:6>: '',
 <Page:7>: '',
 <Page:8>: 'опылителей',
 <Page:9>: 'Зоология, морфология, анатомия, физиология, экология, пале\xad\nонтология, этология, опылители.',
 <Page:10>: 'среды жизни\nНаземно-воздушную среду\nЛ).\nводной среде\nБ).\nпланктон), (нектон),\nбентос)',
 <Page:11>: '',
 <Page:12>: 'Почва\nВ).\nОрганизм\nГ).\nместами обитания',
 <Page:13>: 'абиотических экологи\xad\nческих факторов,\nбиотические экологические факторы.\nантропогенными экологическими фак\xad\nторами.\nсредой оби\xad\nтания.',
 <Page:14>: 'хищничеством.\nШ\nГ,\nпаразитами\nхозяевами.\nпаразитизмом.\nконкурентные отношения',
 <Page:15>: 'биоценозы.\nпи\xad\nщевые связи:\nцепи питания',
 <Page:16>: 'Среды жизни (наземно-воздушная, водная, почва, организм),\nместа оби

In [ ]:
# Минимально подчистим
dict_word_prep = {key:dict_word[key].replace('\xad\n', '') for key in dict_word}
dict_word_prep = {key:dict_word_prep[key].replace('\n', ' ') for key in dict_word_prep}

In [ ]:
dict_word_prep

{<Page:1>: '',
 <Page:2>: 'издание, переработанное',
 <Page:3>: '',
 <Page:4>: 'полужирным курсивом.',
 <Page:5>: 'Глава 1 зоологией зоон логос морфология анатомия животных Физиология животных экология животных, Палеонтология Этология',
 <Page:6>: 'А Б —',
 <Page:7>: '',
 <Page:8>: 'опылителей',
 <Page:9>: 'Зоология, морфология, анатомия, физиология, экология, палеонтология, этология, опылители.',
 <Page:10>: 'среды жизни Наземно-воздушную среду Л). водной среде Б). планктон), (нектон), бентос)',
 <Page:11>: 'А Б В',
 <Page:12>: 'Почва В). Организм Г). местами обитания',
 <Page:13>: 'абиотических экологических факторов, биотические экологические факторы. антропогенными экологическими факторами. средой обитания.',
 <Page:14>: 'хищничеством. Ш 1 3 Г, паразитами хозяевами. паразитизмом. конкурентные отношения',
 <Page:15>: '1 2 3 биоценозы. пищевые связи: цепи питания',
 <Page:16>: 'Среды жизни (наземно-воздушная, водная, почва, организм), места обитания, экологические факторы (биотически

In [ ]:
list(dict_word_prep.items())[8]

(<Page:9>,
 'Зоология, морфология, анатомия, физиология, экология, палеонтология, этология, опылители.')

# **Pandas**

## 1) DataFrame из данных, обработанных с помощью модели **sage-fredt5-large**

In [ ]:
df_ch_1 = pd.DataFrame(ch_1_result_fin)

In [ ]:
df_ch_1.columns = ['Текст'] # переименуем колонку

In [ ]:
df_ch_1 = df_ch_1.join(pd.DataFrame(
    {
        "Учебник": "Биология 7 кл. Учебник для учащихся общеобразовательных организаций В.М. Константинов, В.Г. Бабенко, B.C. Кучменко. — 5-е изд., перераб. — М. ;  Вентана-Граф 2016",
        "Предмет": "Биология",
        "Класс" : 7,
        "Глава" : title_list0[0],
        "Параграф" : chapter_list0[0],
        "Ключевые слова" : list(dict_word_prep.values())[8]
        }, index=df_ch_1.index
    ))

In [ ]:
df_ch_1

,Текст,Учебник,Предмет,Класс,Глава,Параграф,Ключевые слова
0,Глава 1 ____________ О б и е с в е д е н и я о...,Биология 7 кл. Учебник для учащихся общеобразо...,Биология,7,Глава 1. Общие сведения о мире животных,§ 1. Зоология — наука о ж и вотн ы х,"Зоология, морфология, анатомия, физиология, эк..."


In [ ]:
# Поменяем местами колонки
cols = list(df_ch_1.columns)
a, b, c, d, e, f, g = cols.index('Текст'), cols.index('Учебник'), cols.index('Предмет'), cols.index('Класс'), cols.index('Глава'), cols.index('Параграф'), cols.index('Ключевые слова')
cols[a], cols[b], cols[c], cols[d], cols[e], cols[f], cols[g] = cols[b], cols[c], cols[d], cols[e], cols[f], cols[a], cols[g]
df_ch_1 = df_ch_1[cols]

In [ ]:
df_ch_1

,Учебник,Предмет,Класс,Глава,Параграф,Текст,Ключевые слова
0,Биология 7 кл. Учебник для учащихся общеобразо...,Биология,7,Глава 1. Общие сведения о мире животных,§ 1. Зоология — наука о ж и вотн ы х,Глава 1 ____________ О б и е с в е д е н и я о...,"Зоология, морфология, анатомия, физиология, эк..."


In [ ]:
df_ch_1.to_excel("Chapter_1_prep.xlsx") # сохраняем в эксель

## 2) DataFrame из данных, обработанных с помощью модели **sage-fredt5-distilled-95m**

In [ ]:
df_ch_1_95 = pd.DataFrame(ch_1_result_95_fin)

In [ ]:
df_ch_1_95.columns = ['Текст'] # переименуем колонку

In [ ]:
df_ch_1_95 = df_ch_1_95.join(pd.DataFrame(
    {
        "Учебник": "Биология 7 кл. Учебник для учащихся общеобразовательных организаций В.М. Константинов, В.Г. Бабенко, B.C. Кучменко. — 5-е изд., перераб. — М. ;  Вентана-Граф 2016",
        "Предмет": "Биология",
        "Класс" : 7,
        "Глава" : title_list0[0],
        "Параграф" : chapter_list0[0],
        "Ключевые слова" : list(dict_word_prep.values())[8]
        }, index=df_ch_1_95.index
    ))

In [ ]:
df_ch_1_95

,Текст,Учебник,Предмет,Класс,Глава,Параграф,Ключевые слова
0,Глава 1. Об общ. ие с в е д е н и я о м и р е ...,Биология 7 кл. Учебник для учащихся общеобразо...,Биология,7,Глава 1. Общие сведения о мире животных,§ 1. Зоология — наука о ж и вотн ы х,"Зоология, морфология, анатомия, физиология, эк..."


In [ ]:
# Поменяем местами колонки
cols_95 = list(df_ch_1_95.columns)
a_95, b_95, c_95, d_95, e_95, f_95, g_95 = cols_95.index('Текст'), cols_95.index('Учебник'), cols_95.index('Предмет'), cols_95.index('Класс'), cols_95.index('Глава'), cols_95.index('Параграф'), cols_95.index('Ключевые слова')
cols_95[a_95], cols_95[b_95], cols_95[c_95], cols_95[d_95], cols_95[e_95], cols_95[f_95], cols_95[g_95] = cols_95[b_95], cols_95[c_95], cols_95[d_95], cols_95[e_95], cols_95[f_95], cols_95[a_95], cols_95[g_95]
df_ch_1_95 = df_ch_1_95[cols_95]

In [ ]:
df_ch_1_95

,Учебник,Предмет,Класс,Глава,Параграф,Текст,Ключевые слова
0,Биология 7 кл. Учебник для учащихся общеобразо...,Биология,7,Глава 1. Общие сведения о мире животных,§ 1. Зоология — наука о ж и вотн ы х,Глава 1. Об общ. ие с в е д е н и я о м и р е ...,"Зоология, морфология, анатомия, физиология, эк..."


In [ ]:
df_ch_1_95.to_excel("Chapter_1_prep_95.xlsx") # сохраняем в эксель